## Overview

This code generates a processed trip dataset that includes both timestamp information and area identifiers for each taxi trip. It also produces a separate dataset with driver information, enabling downstream analysis that integrates trips, drivers, and spatial patterns.

## Data Source

All raw trip records are obtained from the official NYC Taxi & Limousine Commission (TLC) repository:

- NYC TLC Trip Record Data
https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page

## What This Code Does

- Loads raw TLC trip data.

- Cleans and filters trip records (e.g., invalid coordinates, missing timestamps).

- Assigns area IDs (taxi zones) for pickups and dropoffs using previously processed zone data.

- Generates a driver dataset with relevant attributes (e.g., driver IDs, trip counts, working hours).

- Saves all processed outputs as CSV files for use in downstream modeling tasks.


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as n
from math import *
import datetime as dt

# from learnweight import ComputeWeight
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import pyarrow.parquet as pq

## Generate trip dataset with time and area id. 

In [2]:
trips = pq.read_table("..//Database//NYC_trip//yellow_tripdata_2022-06.parquet")
trips = trips.to_pandas()
trips

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2022-06-01 00:25:41,2022-06-01 00:48:22,1.0,11.00,1.0,N,70,48,1,32.00,3.00,0.5,2.00,6.55,0.3,44.35,2.5,0.00
1,1,2022-06-01 00:44:40,2022-06-01 01:01:48,1.0,4.20,1.0,N,170,226,1,14.00,3.00,0.5,0.00,0.00,0.3,17.80,2.5,0.00
2,2,2022-06-01 00:23:07,2022-06-01 00:39:50,1.0,9.49,1.0,N,264,113,1,26.00,0.50,0.5,5.00,6.55,0.3,42.60,2.5,1.25
3,1,2022-06-01 00:25:53,2022-06-01 00:57:06,2.0,12.10,1.0,N,132,17,2,37.00,1.75,0.5,0.00,0.00,0.3,39.55,0.0,1.25
4,1,2022-06-01 00:23:58,2022-06-01 00:33:43,0.0,1.80,1.0,N,140,163,1,9.00,3.00,0.5,2.55,0.00,0.3,15.35,2.5,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3558119,1,2022-06-30 23:45:51,2022-06-30 23:51:48,NaN,0.00,NaN,None,148,256,0,9.20,0.50,0.5,0.00,0.00,0.3,15.00,NaN,NaN
3558120,2,2022-06-30 23:25:00,2022-06-30 23:40:00,NaN,5.01,NaN,None,79,262,0,18.86,0.00,0.5,5.19,0.00,0.3,27.35,NaN,NaN
3558121,2,2022-06-30 23:29:00,2022-06-30 23:37:00,NaN,1.55,NaN,None,164,79,0,10.03,0.00,0.5,3.10,0.00,0.3,16.43,NaN,NaN
3558122,2,2022-06-30 23:24:15,2022-06-30 23:50:19,NaN,5.30,NaN,None,211,239,0,24.34,0.00,0.5,0.00,0.00,0.3,27.64,NaN,NaN


In [3]:
trips = trips[
    ["PULocationID", "tpep_pickup_datetime", "DOLocationID", "tpep_dropoff_datetime"]
]
trips.columns = ["sid", "call_time", "eid", "end_time"]

In [4]:
area_list = pd.read_csv("..//Database//NYC_area//NY_area_List.csv", index_col=0)
trips = trips[trips["sid"].isin(area_list.oxmid)]
trips = trips[trips["eid"].isin(area_list.oxmid)]
trips.to_csv("..//Database//NYC_trip//order_263.csv")
area_list.reset_index(drop=True, inplace=True)
area_list.reset_index(inplace=True)
trips = trips.merge(area_list, left_on="sid", right_on="oxmid")
trips = trips.merge(area_list, left_on="eid", right_on="oxmid")
trips.drop(["oxmid_x", "oxmid_y"], axis=1, inplace=True)
trips.columns = ["sid", "call_time", "eid", "end_time", "sid_260", "eid_260"]
trips = trips[['sid_260', 'call_time', 'eid_260', 'end_time']]
trips.columns = ["sid", "call_time", "eid", "end_time"]
trips.to_csv("..//Database//NYC_trip//order_260.csv")

In [36]:
area_list = pd.read_csv("..//Database//NYC_area//NY_area_List.csv", index_col=0)
area_list = area_list.reset_index(drop=True).reset_index()
np.random.seed(0)
driver = pd.DataFrame({"id": np.random.choice(area_list.oxmid, size=2000)})
driver.to_csv("..//Database//NYC_trip//driver_263.csv")
driver = driver.merge(area_list, left_on="id", right_on="oxmid")
driver.drop(["oxmid"], axis=1, inplace=True)
driver.columns = ["id_263", "id_260"]
driver[['id_260']].to_csv("..//Database//NYC_trip//driver_260_2000.csv")
driver[['id_263']].to_csv("..//Database//NYC_trip//driver_263_2000.csv")

In [10]:
order = trips
order["call_time"] = pd.to_datetime(order["call_time"])
order["end_time"] = pd.to_datetime(order["end_time"])
start_time = pd.to_datetime("2022-06-01 06:00:00 AM")
end_time = pd.to_datetime("2022-06-01 07:00:00 AM")
order_pick = order[(order["call_time"] > start_time) & (order["call_time"] <= end_time)]
order_pick = order_pick[["sid", "call_time", "eid", "end_time"]]
order_pick["sid"] -= 1
order_pick["eid"] -= 1
order_pick = order_pick[(order_pick["sid"] <= 262) & (order_pick["eid"] <= 262)]